In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from textwrap import shorten
import requests

from bs4 import BeautifulSoup

In [2]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [3]:
df = pd.read_csv('Miami_River_Map - Stories_after_and_during_Nov2022.csv')

In [4]:
df['geo_address'] = df['full_address'] + ' Miami, Florida'

In [5]:
df

,full_address,Reporter_Outreach,developers,description,condos_or_apts,status,price_paid_for_property,construction_loan_amount,lender_name,anchor_tenant?,commercial_sq_ft,anchor_tenant_leaseterm,story_link,geo_address
0,99 Southwest Seventh Street in Miami,Call (Lidia),Newgard Development Group,"Newgard Development Group, led by Hernandez, p...",Roughly 869 (This is a low-balled figure but s...,"Plans filed, no date given","$57,500,000",NaN,NaN,NaN,"1,100",NaN,https://therealdeal.com/miami/2023/10/31/harve...,"99 Southwest Seventh Street in Miami Miami, Fl..."
1,1411 Northwest North River Drive,Checking,Urban-X Group,"Urban-X Group, led by Andrew Hellinger and Cor...",Apartments (475),"Proposed, awaiting board approval","$13,200,000",NaN,NaN,NaN,"360,000",NaN,https://therealdeal.com/miami/2023/09/19/urban...,"1411 Northwest North River Drive Miami, Florida"
2,400 Southeast Second Avenue,XX,"Hyatt, Gencom",Gencom and Hyatt plan a pair of 61-story tower...,Apartments (1806) and 615-key Hyatt hotel if w...,"Lease negotiated, finalizing construction plans",Rented from Miami City,NaN,NaN,NaN,NaN,NaN,https://therealdeal.com/miami/2023/07/17/hyatt...,"400 Southeast Second Avenue Miami, Florida"
3,444 Brickell Avenue,XX,Related,Coconut Grove-based Related is developing a 75...,Condos (355) and Apartments (506),Baccarat-branded condos and 506-unit apartment...,NaN,"$164,000,000",NaN,NaN,NaN,NaN,https://therealdeal.com/miami/2023/07/11/miami...,"444 Brickell Avenue Miami, Florida"
4,"2750 N.W. South River Drive, as well as 1701 a...",Call (Lidia),ROVR,Oscar Rodriguez and Ricardo Vadia through thei...,"1,593 apartments",Construction to start in the coming months of ...,NaN,NaN,NaN,NaN,NaN,NaN,https://therealdeal.com/miami/2023/06/06/rovr-...,"2750 N.W. South River Drive, as well as 1701 a..."
5,1451 Northwest 14th Street,Call (Lidia),Alta Development,"Alta Development, founded and led by Henry Pin...",283 condos,Developer has applied for building permit,NaN,NaN,NaN,NaN,NaN,NaN,https://therealdeal.com/miami/2023/04/24/miami...,"1451 Northwest 14th Street Miami, Florida"
6,24 Southwest Fourth Street,Call (Adam),"Fortune International Group, KAR Properties","Miami-based Fortune International Group, led b...",condos (386),"Planned, financing secured","$59,000,000","$38,000,000",Bank OZK,NaN,NaN,NaN,https://therealdeal.com/miami/2023/01/05/fortu...,"24 Southwest Fourth Street Miami, Florida"
7,501 Southwest Third Avenue,call (Adam),Chetrit Group,"The Chetrit Group plans a sprawling, multi-pha...",Resi units (1600) (one tower will have 378 con...,NaN,"$91,000,000","$325,000,000","Madison Realty Capital (310), MSD Partners (15)",NaN,NaN,NaN,https://therealdeal.com/miami/2022/12/14/chetr...,"501 Southwest Third Avenue Miami, Florida"
8,1250 Northwest 22nd Avenue,call (Adam),NaN,The Polish American Club of Miami has requeste...,102 apartments,Continued indefinitely by the Miami City Comis...,NaN,NaN,NaN,NaN,NaN,NaN,https://therealdeal.com/miami/2022/11/01/polis...,"1250 Northwest 22nd Avenue Miami, Florida"
9,3550 Northwest South River Drive,XX,Coastline-Nakash Equity Capital Group,"Yaniv Nakash and Eyal Mehaber, who led Coastli...",822 apartments,Planned,"$25,000,000",NaN,NaN,NaN,"60,000",NaN,https://therealdeal.com/miami/2023/09/22/coast...,"3550 Northwest South River Drive Miami, Florida"


In [6]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Image URL Collector

In [8]:
df['image_links'] = ''
links = []
for url in df['story_link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find('img',{'class':'MainArtImage_image__2YtGm'})
        images = images['src']
        links.append(images)
    except:
        links.append('no_image')

df['image_links'] = links

In [9]:
print(df['story_link'].iloc[0])

https://therealdeal.com/miami/2023/10/31/harvey-hernandezs-newgard-plans-condo-tower-in-brickell/


In [10]:
# df.at[20,'image_img']=('<img src="Aimco.jpg" width="256" height="156">')
print(df['image_links'].iloc[1])

https://static.therealdeal.com/wp-content/uploads/2023/09/Urban-X-Group-Plans-Rentals-as-Part-of-River-Landing-Project-m-1400x875.jpg


In [11]:
# df.at[4,'image_links']=('https://therealdeal.com/wp-content/uploads/2019/10/trd-logo-16x9.jpg')

In [12]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0     <a href="https://therealdeal.com/miami/2023/10...
1     <a href="https://therealdeal.com/miami/2023/09...
2     <a href="https://therealdeal.com/miami/2023/07...
3     <a href="https://therealdeal.com/miami/2023/07...
4     <a href="https://therealdeal.com/miami/2023/06...
5     <a href="https://therealdeal.com/miami/2023/04...
6     <a href="https://therealdeal.com/miami/2023/01...
7     <a href="https://therealdeal.com/miami/2022/12...
8     <a href="https://therealdeal.com/miami/2022/11...
9     <a href="https://therealdeal.com/miami/2023/09...
10    <a href="https://therealdeal.com/miami/2023/05...
11    <a href="https://therealdeal.com/miami/2022/01...
Name: description_link, dtype: object

In [13]:
df.columns

Index(['full_address', 'Reporter_Outreach', 'developers', 'description',
       'condos_or_apts', 'status', 'price_paid_for_property',
       'construction_loan_amount', 'lender_name', 'anchor_tenant?',
       'commercial_sq_ft', 'anchor_tenant_leaseterm', 'story_link',
       'geo_address', 'geocoded', 'lat', 'lon', 'image_links',
       'short_description', 'remaining_desc', 'description_link'],
      dtype='object')

In [14]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    developer = df['developers'].iloc[i]
    image = df['image_links'].iloc[i]
    description = df['description_link'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Developer: </strong>{}'''.format(developer) + '''<br>
    {}'''.format(description) + '''<br>
    </html>
    '''
    return html

In [15]:
import branca

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=13)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Miami River Development Projects')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [16]:
m.save('index.html')

## Map URL Snagger

In [17]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiRiver_11_7_23_resi_dev
